In [117]:
import os
import sys
import csv
import json

import numpy as np
import pandas as pd
import scipy.stats as stats

import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from collections import defaultdict

from scipy.optimize import curve_fit
from scipy.special import factorial
from scipy.stats import poisson
from scipy.stats import nbinom
from scipy.stats import chi2

%matplotlib inline

In [118]:
minimum_maf = 0.01

# Load the permuation/randomization study results

In [119]:
# we did an analysis to compare the maf distribution
# this makes sure we grab only those that are similar

maf_distribution_stats_files = (
    ('gs2', 'data/ukbb/gene_set_2/gene_set_2.json'),
)

run2excluded_samples = dict()

for run_code, maf_stats_file in maf_distribution_stats_files:
    fh = open(maf_stats_file)
    results = json.load(fh)
    fh.close()
    
    excluded_samples = [r['name'].strip('.txt') for r in results['sample_stats'] if r['Up'] < 0.05]
    run2excluded_samples[run_code] = excluded_samples
    
    del excluded_samples
    

In [130]:
random_sample_files = (
    # gene_set_kegg
    ('kegg_apr_snp', 'data/ukbb/gene_set_kegg/ukbb_plinkresults.old_genesnpssamples_1.txt'),
    ('kegg_apr_phe', 'data/ukbb/gene_set_kegg/ukbb_phenpermuteresults.txt'),
    # Gene Set 2
    ('gs2_apr_snp', 'data/ukbb/gene_set_2/ukbb_plinkresults_genesnpssamples_2_pos_severe_apr_bydist_0to80kb.txt'),
    ('gs2_may_snp', 'data/ukbb/gene_set_2/ukbb_plinkresults_genesnpssamples_2_pos_severe_may_bydist_0to80kb.txt'),
    ('gs2_apr_phe', 'data/ukbb/gene_set_2/ukbb_phenpermuteresults_pos_severe_apr_snps_gene_2_bydist_0to80kb.txt'),
    ('gs2_may_phe', 'data/ukbb/gene_set_2/ukbb_phenpermuteresults_pos_severe_may_snps_gene_2_bydist_0to80kb.txt'),
    # Gene Set LD
    ('ld_apr_phe', 'data/ukbb/gene_set_ld/ukbb_phenpermuteresults_pos_severe_apr_snps_gene_ld_bydist_0to80kb.txt'),
    ('ld_may_phe', 'data/ukbb/gene_set_ld/ukbb_phenpermuteresults_pos_severe_may_snps_gene_ld_bydist_0to80kb.txt')
)

run2dist2alpha2sig = dict()
run2output_file_prefix = dict()

for run_code, random_sample_file in random_sample_files:

    run2output_file_prefix[run_code] = random_sample_file.split('/')[-1].split('.')[0]
    
    excluded_samples = run2excluded_samples.get(run_code.split('_')[0], list())
    print("May exclude up to %d samples." % len(excluded_samples))
    
    fh = open(random_sample_file)
    reader = csv.reader(fh, delimiter='\t')
    header = next(reader)

    dist2alpha2sig = defaultdict(lambda: defaultdict(list))
    alphas = set()
    
    excluded_set = set()
    for row in reader:
        if len(row) == 6:
            sid, maf, dist, alpha, numvar, sigvar = row
        elif len(row) == 4:
            sid, alpha, numvar, sigvar = row
            dist = 60000
            maf = 0.001
        else:
            raise Exception("Row is unexpected length.")
        
        if float(maf) != minimum_maf:
            continue
        
        if sid in excluded_samples:
            excluded_set.add(sid)
            continue
        
        dist2alpha2sig[int(dist)][float(alpha)].append(int(sigvar))
        alphas.add(float(alpha))
    
    alphas = sorted(alphas)
    run2dist2alpha2sig[run_code] = dist2alpha2sig
    
    print(" Excluded %d samples." % len(excluded_set))
    del dist2alpha2sig
    fh.close()

May exclude up to 0 samples.
 Excluded 0 samples.
May exclude up to 0 samples.
 Excluded 0 samples.
May exclude up to 52 samples.
 Excluded 52 samples.
May exclude up to 52 samples.
 Excluded 52 samples.
May exclude up to 52 samples.
 Excluded 0 samples.
May exclude up to 52 samples.
 Excluded 0 samples.
May exclude up to 0 samples.
 Excluded 0 samples.
May exclude up to 0 samples.
 Excluded 0 samples.


## Load the Association Study results

In [131]:
plink_results_files = (
    ('gs2_apr', 'data/ukbb/gene_set_2/apr_pos_severe.PHENO1.glm.logistic.adjusted'),
    ('gs2_may', 'data/ukbb/gene_set_2/may_pos_severe.PHENO1.glm.logistic.adjusted'),
    ('ld_apr', 'data/ukbb/gene_set_ld/apr_pos_severe.PHENO1.glm.logistic.adjusted'),
    ('ld_may', 'data/ukbb/gene_set_ld/may_pos_severe.PHENO1.glm.logistic.adjusted')
)

run2plink_results = dict()

for run_code, plink_results_file in plink_results_files:

    fh = open(plink_results_file)
    reader = csv.reader(fh, delimiter='\t')

    header = next(reader)
    
    plink_results = list()

    for row in reader:
        if row[0][0] == '#':
            continue

        chrom = row[0]
        rsid = row[1]
        allele = row[2]
        unadjusted = float(row[3])
        fdrbh = float(row[9])

        if True or snp_allele_counts[rsid]['maf'] > minimum_maf:
            plink_results.append([chrom, rsid, allele, unadjusted, fdrbh])
        
    run2plink_results[run_code] = plink_results
    
    del plink_results
    fh.close()

In [132]:
# load the distances for the actual results
plink_distance_files = (
    ('gs2', 'data/ukbb/gene_set_2/05172020_snps_gene_distance.txt'),
    ('ld', 'data/ukbb/gene_set_ld/05242020_snps_gene_distance.txt')
)

run2plink_rsid2distance = dict()

for run_code, plink_results_distance_file in plink_distance_files:
    
    fh = open(plink_results_distance_file)
    reader = csv.reader(fh, delimiter='\t')

    header = next(reader)

    plink_rsid2distance = dict()
    for rsid, closest_gene, distance in reader:
        plink_rsid2distance[rsid] = int(distance)
    
    run2plink_rsid2distance[run_code] = plink_rsid2distance
    
    del plink_rsid2distance
    fh.close()

## Evaluate the results

In [133]:
# dist = 40000

# pvalues = [row[3] for row in plink_results if plink_rsid2distance[row[1]] < dist]

# x, y, ylo, yhi = zip(*[(a, np.percentile(s, 50), np.percentile(s, 50)-np.percentile(s, 2.5), np.percentile(s, 97.5)-np.percentile(s, 50)) for a, s in dist2alpha2sig[dist].items()])
# plt.errorbar(x, y, yerr=(ylo, yhi), fmt='ko', lw=1)

# y_act = [len([p for p in pvalues if p <= a]) for a in x]
# plt.plot(x, y_act, 'rd')


# plt.xscale('log')
# plt.yscale('log')
# sns.despine(trim=False)


In [134]:
# run this the first time only, the next part can take a while and this will keep the results in memory
dist2run2results = dict()

In [135]:
distribution_functions = [
    {
        'name': 'chi-squared',
        'pdf': chi2.pdf,
        'cdf': chi2.cdf,
        'mean': chi2.mean,
        'nparams': 1,
        'p0': [2,],
    },
    {
        'name': 'negative-binomial',
        'pdf': nbinom.pmf,
        'cdf': nbinom.cdf,
        'mean': nbinom.mean,
        'nparams': 2,
        'p0': [1, 0.01]
    },
    {
        'name': 'poisson',
        'pdf': poisson.pmf,
        'cdf': poisson.cdf,
        'mean': poisson.mean,
        'nparams': 1,
        'p0': [0.3,],
    }
]

In [ ]:
for dist_func in distribution_functions:
    
    #print(dist_func)
    
    fit_function = dist_func['pdf']
    p0 = dist_func['p0']
    cdf = dist_func['cdf']
    
    run2results = dist2run2results.get(dist_func['name'], dict())
    
    for plink_run_code in run2plink_results.keys():
        
        for perm_method in ('snp', 'phe'):

            random_run_code = "%s_%s" % (plink_run_code, perm_method)
            print(dist_func['name'], random_run_code)
            
            # check if we already computed the results for this combo
            if not random_run_code.startswith('ld') and random_run_code in run2results:
                continue
            
            # make sure we have the data we need to run this combo
            if not random_run_code in run2dist2alpha2sig:
                continue
            
            dist_run_code = plink_run_code.split('_')[0]

            #print(plink_run_code, random_run_code, dist_run_code)

            actual_pvalues = [(row[3], run2plink_rsid2distance[dist_run_code][row[1]]) for row in run2plink_results[plink_run_code]]

            results = list()
            for dist, alpha2sig in tqdm(run2dist2alpha2sig[random_run_code].items()):

                for alpha, sigcs in alpha2sig.items():
                    #print(alpha)

                    if max(sigcs) <= 1:
                        continue

                    # how many actual hits are there at this alpha?
                    nhits = len([p for p,d in actual_pvalues if p <= alpha and d <= dist])

                    plt.figure(figsize=(2,2))
                    # the bins should be of integer width, because neg bin is an integer distribution
                    bins = np.arange(max(sigcs)+2) - 0.5
                    entries, bin_edges, patches = plt.hist(sigcs, bins=bins, density=True, label='Random', color='k')

                    # calculate bin centres
                    bin_middles = 0.5 * (bin_edges[1:] + bin_edges[:-1])

                    # fit with curve_fit
                    parameters, cov_matrix = curve_fit(fit_function, bin_middles, entries, p0=p0, maxfev = 6000)
                    #print(parameters)

                    # plot with fitted parameters
                    x_plot = np.arange(0, max(max(sigcs)+2,nhits))

                    plt.plot(
                        x_plot,
                        fit_function(x_plot, *parameters),
                        marker='o', linestyle='',
                        color='r',
                        label='Neg Bin Fit',
                    )
                    plt.ylabel('Density')
                    plt.xlabel('# Rejected H0')
                    plt.title(r'$\alpha = %.2e$' % alpha)

                    sigcounts = dict()
                    for c in set(sigcs):
                        sigcounts[c] = len([x for x in sigcs if x == c])

                    actual_counts = [sigcounts[c] for c in sorted(sigcounts)]
                    expected_counts = len(sigcs)*fit_function(list(map(int,sorted(sigcounts))), *(parameters))

                    if min(expected_counts) == 0:
                        expected_counts += 1

                    chi, chipvalue = stats.chisquare(
                        actual_counts, 
                        expected_counts
                    )

                    # p-value for the actual number of hits
                    nhits_pvalue = 1-cdf(nhits, *parameters)

                    plt.plot([nhits, nhits], [0, max(entries)], 'r--')
                    sns.despine(trim=True)

                    plt.close()
                    results.append({
                        'distance': dist, 
                        'alpha': alpha, 
                        'num_reps': len(sigcs), 
                        'max_rep_hits': max(sigcs), 
                        'fit_p': list(parameters),
                        'chisquared': chi, 
                        'chi_pvalue': chipvalue, 
                        'nhits': nhits, 
                        'nhits_pvalue': nhits_pvalue,
                        'actual_counts': actual_counts,
                        'expected_counts': list(expected_counts)})

            run2results[random_run_code] = results
    
    dist2run2results[dist_func['name']] = run2results
    del run2results



In [137]:
dist2run2results.keys()

dict_keys(['chi-squared', 'negative-binomial', 'poisson'])

In [138]:
# Save dist2run2results for later plotting
fh = open('results/null_distirbution_fit_results_minmaf%.3f_0to80kb.json' % minimum_maf, 'w')
fh.write(json.dumps(dist2run2results))
fh.close()

In [150]:
# determine which alphas to plot automatically from the data
alphas = sorted(set([r['alpha'] for r in dist2run2results['negative-binomial']['gs2_apr_phe']]))
alphas.reverse()

# manual set which alphas to plot
# alphas = [0.005, 0.0025, 0.001, 0.0005]

In [151]:
# pretty pastel-like colors
colors = sns.color_palette("cubehelix", n_colors=len(alphas))

# manually defined colors
# colors = ['#DDCA70', '#235555', '#4198C7', '#464162']

In [167]:
ylimits = {
    'negative-binomial': {
        'gs2_apr_snp': {
            'y': (10e-4, 1.),
            'x': (40000, 80000),
        },
        'gs2_apr_phe': {
            'y': (10e-7, 1.),
            'x': (40000, 80000),
        },
        'gs2_may_snp': {
            'y': (10e-4, 1.),
            'x': (40000, 80000),
        },
        'gs2_may_phe': {
            'y': (10e-7, 1.),
            'x': (40000, 80000),
        },
        'ld_apr_phe': {
            'y': (10e-5, 1.),
            'x': (0, 20000),
        },
        'ld_may_phe': {
            'y': (10e-5, 1.),
            'x': (0, 20000),
        }
    }
}

In [ ]:
for dist_name, run2results in dist2run2results.items():
    for run_code, results in run2results.items():

        sns.set(style='ticks', font_scale=1.2)
        plt.figure(figsize=(6,4))

        

        for i, a in enumerate(alphas):
            results_for_alpha = [(r['distance'], r['nhits_pvalue'], r['chi_pvalue']) for r in results if r['alpha']==a]
            
            if len(results_for_alpha) == 0:
                continue

            x, y, gofs = zip(*results_for_alpha)
            
            pinsig_gofs = len([g for g in gofs if g >= 0.05])/len(gofs)
#             plt.plot(x,y, label=r'$\alpha = %s (%.2f)$' % (a, pinsig_gofs), color=colors[i])
            plt.plot(x,y, label=r'$\alpha = %s$' % a, color=colors[i], lw=3)
            plt.yscale('log')
            plt.ylabel('Significance of Number of Hits')
            plt.xlabel('Distance to Nearest Gene')
            
            if dist_name in ylimits:
                plt.ylim(ylimits[dist_name][run_code]['y'])
                plt.xlim(ylimits[dist_name][run_code]['x'])

        plt.plot([0, 80000], [0.05, 0.05], 'k--', label=r'$P=0.05%$')
        plt.legend(bbox_to_anchor=(1, 1), loc='upper left', ncol=1)
        plt.title(run_code + ' ' + dist_name)

        sns.despine()
        plt.tight_layout()
        plt.savefig('figures/r1/permutation-analysis/%s_sighits_%s_minmaf%.3f_0to80kb.pdf' % (run2output_file_prefix[run_code], dist_name, minimum_maf))
        
        

In [ ]:
dist_name = 'negative-binomial'
run2results = dist2run2results[dist_name]

for run_code, results in run2results.items():
    
    sns.set(style='ticks', font_scale=1.2)
    plt.figure(figsize=(6,4))

    for i, a in enumerate(alphas):
        results_for_alpha = [(r['distance'], r['nhits']) for r in results if r['alpha']==a]
        if len(results_for_alpha) == 0:
            continue

        x, y = zip(*results_for_alpha)
        plt.plot(x,y, label=r'$\alpha = %s$' % a, color=colors[i], lw=3)
    #     plt.yscale('log')
        plt.ylim(0,45)
        plt.ylabel('Number of Hits')
        plt.xlabel('Distance to Nearest Gene')

    plt.legend(bbox_to_anchor=(1, 1), loc='upper left', ncol=1)
    plt.title(run_code)

    sns.despine()
    plt.tight_layout()
    plt.savefig('figures/r1/permutation-analysis/%s_numhits_minmaf%.3f_0to80kb.pdf' % (run2output_file_prefix[run_code], minimum_maf))


In [161]:
alphas = sorted(set([r['alpha'] for r in dist2run2results['negative-binomial']['gs2_apr_phe']]))
alphas.reverse()

# pretty pastel-like colors
colors = sns.color_palette("cubehelix", n_colors=len(alphas))